First 3 parts of Assignment

In [15]:
import tensorflow as tf
import tensorflow_probability as tfp
import matplotlib.pyplot as plt
tfd = tfp.distributions
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers

Part-1<br>
Sampling the source Distribution

In [2]:
def uniformSampler(support, dims):
  '''
  support is a list which contains info of support of distribution
  dims : dimension of r.v
  '''
  low = []
  high = []
  for i in range(dims):
    low.append(support[0])
    high.append(support[1])

  U = tfd.Uniform(low = low,high = high)
  IndependentUniform = tfd.Independent(U, reinterpreted_batch_ndims=1)

  return IndependentUniform

In [3]:
sampler = uniformSampler([1,2],2)

In [4]:
sampler.sample(1).numpy().squeeze()

array([1.2342336, 1.2992909], dtype=float32)

Part-2<br>
Sampling the target distribution using GMM

In [5]:
import numpy as np

In [6]:
def GMMsampler(pi,mus,sigmas):
  '''
  pi : probabilities of hidden variables
  mus : means of clusters
  sigmas : std of clusters
  '''
  
  gmm = tfd.Mixture(
    cat = tfd.Categorical(probs = pi),
    components = [tfd.MultivariateNormalDiag(loc = mus[i], scale_diag = sigmas[i]) for i in range(mus.shape[0])] )
  
  return gmm

In [7]:
pi = np.array([0.3,0.7]) # probabilities of hidden variables
mus = np.array([[2.,3.],[1.,4.]])
sigmas = np.array([[1.,1.],[1.,2.]])

In [8]:
gmmSampler = GMMsampler(pi,mus,sigmas)

In [9]:
gmmSampler.sample().numpy()

array([1.21769471, 6.19134892])

Part-3<br>
Sampling using Normalising Flows(RNVP)<br>
$$x_1 = u_1$$
$$x_2 = u_2.\sigma(u_1) + \mu(u_1)$$

In [70]:
class NN(layers.Layer):

    """
    Neural Network Architecture for calcualting s and t for Real-NVP
    
    :param input_shape: shape of the data coming in the layer
    :param hidden_units: Python list-like of non-negative integers, specifying the number of units in each hidden layer.
    :param activation: Activation of the hidden units
    """
    
    def __init__(self, input_shape, n_hidden=[64,64], activation="relu"):
        super(NN, self).__init__(name="nn")
        layer_list = []
        for n in n_hidden:
            layer_list.append(layers.Dense(n, activation=activation))
        self.layer_list = layer_list
        self.log_s_layer = layers.Dense(input_shape, activation="tanh", name='log_s')
        self.t_layer = layers.Dense(input_shape, name='t')

    def call(self, x):
        y = x
        for layer in self.layer_list:
            y = layer(y)
        log_s = self.log_s_layer(y)
        t = self.t_layer(y)
        return log_s, t

In [71]:
neuralNet = NN(1)

In [86]:
def NFsampler(neuralNet,n):
  '''
  n: number of samples
  neuralNet: neural network to get scaling and shifting factors
  '''
  
  samples = []
  for i in range(n):
    U = ((uniformSampler([0,1], 2)).sample()).numpy()
    u1,u2 = tf.split(U,2)
    u1 = u1[...,np.newaxis]
    u2 = u2[...,np.newaxis]
    x1 = u1
    log_s, t = neuralNet.call(u1)
    x2 = tf.exp(log_s)*U[1] + t
    samples.append([x1,x2])

  samples = np.asarray(samples).squeeze()
  
  return samples 

In [87]:
samples = NFsampler(neuralNet,5)

In [88]:
for sample in samples:
  print(sample)

[0.88382375 0.47093803]
[0.6752194  0.34654188]
[0.19863641 0.6859551 ]
[0.42696834 0.10517609]
[0.06880498 0.04574803]
